![title](imgs/97.png)

![title](imgs/98.png)

In [1]:
from collections import namedtuple
from datetime import datetime

In [2]:
def cast(data_type,value,*,default='N/A'):
    if data_type == 'STRING':
        return str(value)
    elif data_type == 'INT':
        try:
            return int(value)
        except ValueError:
            return default
    else:
        try:
            return datetime.strptime(value,'%m/%d/%Y').date()
        except ValueError:
            return default

In [3]:
file = 'nyc_parking_tickets_extract.csv'
DATA_TYPES = ("INT",'STRING','STRING','STRING','DATE','INT','STRING','STRING','STRING')

In [4]:
def cast_row(data_types,data_row):
    return [cast(data_type,value) for data_type, value in zip(data_types,data_row)]

In [5]:
def nyc_parking_tickets(file):
    with open(file,'r', encoding='utf-8',errors='ignore') as f:
        file_iter = iter(f)
        headers = next(file_iter).strip('\n').split(',')
        headers = [header.replace(' ','_') for header in headers]
        Ticket = namedtuple('Ticket',headers)
        
        tickets = [Ticket(*cast_row(DATA_TYPES,line.strip('\n').split(','))) for line in file_iter]
        return tickets

In [6]:
tickets = nyc_parking_tickets(file)

In [7]:
tickets

[Ticket(Summons_Number=4006478550, Plate_ID='VAD7274', Registration_State='VA', Plate_Type='PAS', Issue_Date=datetime.date(2016, 10, 5), Violation_Code=5, Vehicle_Body_Type='4D', Vehicle_Make='BMW', Violation_Description='BUS LANE VIOLATION'),
 Ticket(Summons_Number=4006462396, Plate_ID='22834JK', Registration_State='NY', Plate_Type='COM', Issue_Date=datetime.date(2016, 9, 30), Violation_Code=5, Vehicle_Body_Type='VAN', Vehicle_Make='CHEVR', Violation_Description='BUS LANE VIOLATION'),
 Ticket(Summons_Number=4007117810, Plate_ID='21791MG', Registration_State='NY', Plate_Type='COM', Issue_Date=datetime.date(2017, 4, 10), Violation_Code=5, Vehicle_Body_Type='VAN', Vehicle_Make='DODGE', Violation_Description='BUS LANE VIOLATION'),
 Ticket(Summons_Number=4006265037, Plate_ID='FZX9232', Registration_State='NY', Plate_Type='PAS', Issue_Date=datetime.date(2016, 8, 23), Violation_Code=5, Vehicle_Body_Type='SUBN', Vehicle_Make='FORD', Violation_Description='BUS LANE VIOLATION'),
 Ticket(Summons

In [8]:
def get_clean_data(file):
    with open(file,'r', encoding='utf-8',errors='ignore') as f:
        headers = next(f).strip('\n').split(',')
        headers = [header.replace(' ','_').lower() for header in headers]
        Ticket = namedtuple('Ticket',headers)
        for row in f:
            yield Ticket(*cast_row(DATA_TYPES,row.strip('\n').split(',')))

In [9]:
list(get_clean_data(file))

[Ticket(summons_number=4006478550, plate_id='VAD7274', registration_state='VA', plate_type='PAS', issue_date=datetime.date(2016, 10, 5), violation_code=5, vehicle_body_type='4D', vehicle_make='BMW', violation_description='BUS LANE VIOLATION'),
 Ticket(summons_number=4006462396, plate_id='22834JK', registration_state='NY', plate_type='COM', issue_date=datetime.date(2016, 9, 30), violation_code=5, vehicle_body_type='VAN', vehicle_make='CHEVR', violation_description='BUS LANE VIOLATION'),
 Ticket(summons_number=4007117810, plate_id='21791MG', registration_state='NY', plate_type='COM', issue_date=datetime.date(2017, 4, 10), violation_code=5, vehicle_body_type='VAN', vehicle_make='DODGE', violation_description='BUS LANE VIOLATION'),
 Ticket(summons_number=4006265037, plate_id='FZX9232', registration_state='NY', plate_type='PAS', issue_date=datetime.date(2016, 8, 23), violation_code=5, vehicle_body_type='SUBN', vehicle_make='FORD', violation_description='BUS LANE VIOLATION'),
 Ticket(summons

### Another Way for Goal 1: 

In [10]:
with open(file) as f:
    column_headers = next(f).strip('\n').split(',')
    sample_data = next(f).strip('\n').split(',')

In [11]:
column_headers

['Summons Number',
 'Plate ID',
 'Registration State',
 'Plate Type',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Violation Description']

In [12]:
def read_data():
    with open(file) as f:
        next(f)
        yield from f

In [13]:
column_names = [header.replace(' ', '_').lower() 
                for header in column_headers]

In [14]:
Ticket = namedtuple('Ticket', column_names)

In [15]:
def parse_int(value, *, default=None):
    try:
        return int(value)
    except ValueError:
        return default

In [16]:
from datetime import datetime
def parse_date(value, *, default=None):
    date_format='%m/%d/%Y'
    try:
        return datetime.strptime(value, date_format).date()
    except ValueError:
        return default

In [17]:
def parse_string(value, *, default=None):
    try:
        cleaned = str(value).strip()
        if not cleaned:
            # empty string
            return default
        else:
            return cleaned
    except ValueError:
        return default

In [18]:
from functools import partial

In [19]:
column_parsers = (parse_int,  # summons_number, default is None
                  parse_string,  # plate_id, default is None
                  partial(parse_string, default=''),  # state
                  partial(parse_string, default=''),  # plate_type
                  parse_date,  # issue_date, default is None
                  parse_int,  # violation_code
                  partial(parse_string, default=''),  # body type
                  parse_string,  # make, default is None
                  lambda x: parse_string(x, default='')  # description
                 )

In [20]:
def parse_row(row, *, default=None):
    fields = row.strip('\n').split(',')
    # note that I'm using a list comprehension here, 
    # since we'll need to iterate through the entire parsed fields
    # twice - one time to check if nothing is None
    # and another time to create the named tuple
    parsed_data = [func(field) 
                   for func, field in zip(column_parsers, fields)]
    if all(item is not None for item in parsed_data):
        print(*parsed_data)
        return Ticket(*parsed_data)
    else:
        return default

In [21]:
rows = read_data()
for _ in range(5):
    row = next(rows)
    parsed_data = parse_row(row)
    print(parsed_data)

4006478550 VAD7274 VA PAS 2016-10-05 5 4D BMW BUS LANE VIOLATION
Ticket(summons_number=4006478550, plate_id='VAD7274', registration_state='VA', plate_type='PAS', issue_date=datetime.date(2016, 10, 5), violation_code=5, vehicle_body_type='4D', vehicle_make='BMW', violation_description='BUS LANE VIOLATION')
4006462396 22834JK NY COM 2016-09-30 5 VAN CHEVR BUS LANE VIOLATION
Ticket(summons_number=4006462396, plate_id='22834JK', registration_state='NY', plate_type='COM', issue_date=datetime.date(2016, 9, 30), violation_code=5, vehicle_body_type='VAN', vehicle_make='CHEVR', violation_description='BUS LANE VIOLATION')
4007117810 21791MG NY COM 2017-04-10 5 VAN DODGE BUS LANE VIOLATION
Ticket(summons_number=4007117810, plate_id='21791MG', registration_state='NY', plate_type='COM', issue_date=datetime.date(2017, 4, 10), violation_code=5, vehicle_body_type='VAN', vehicle_make='DODGE', violation_description='BUS LANE VIOLATION')
4006265037 FZX9232 NY PAS 2016-08-23 5 SUBN FORD BUS LANE VIOLATION

In [22]:
def parsed_data():
    for row in read_data():
        parsed = parse_row(row)
        if parsed: # <---- is not None
            yield parsed

In [23]:
parsed_rows = parsed_data()
for _ in range(5):
    print(next(parsed_rows))

4006478550 VAD7274 VA PAS 2016-10-05 5 4D BMW BUS LANE VIOLATION
Ticket(summons_number=4006478550, plate_id='VAD7274', registration_state='VA', plate_type='PAS', issue_date=datetime.date(2016, 10, 5), violation_code=5, vehicle_body_type='4D', vehicle_make='BMW', violation_description='BUS LANE VIOLATION')
4006462396 22834JK NY COM 2016-09-30 5 VAN CHEVR BUS LANE VIOLATION
Ticket(summons_number=4006462396, plate_id='22834JK', registration_state='NY', plate_type='COM', issue_date=datetime.date(2016, 9, 30), violation_code=5, vehicle_body_type='VAN', vehicle_make='CHEVR', violation_description='BUS LANE VIOLATION')
4007117810 21791MG NY COM 2017-04-10 5 VAN DODGE BUS LANE VIOLATION
Ticket(summons_number=4007117810, plate_id='21791MG', registration_state='NY', plate_type='COM', issue_date=datetime.date(2017, 4, 10), violation_code=5, vehicle_body_type='VAN', vehicle_make='DODGE', violation_description='BUS LANE VIOLATION')
4006265037 FZX9232 NY PAS 2016-08-23 5 SUBN FORD BUS LANE VIOLATION

![title](imgs/99.png)

In [24]:
mapping = {}
for i in parsed_data():
    mapping[i.vehicle_make] = mapping.get(i.vehicle_make,0)+1
for make, count in sorted(mapping.items(),key=lambda t: t[1],reverse=True):
    print(make,count)

4006478550 VAD7274 VA PAS 2016-10-05 5 4D BMW BUS LANE VIOLATION
4006462396 22834JK NY COM 2016-09-30 5 VAN CHEVR BUS LANE VIOLATION
4007117810 21791MG NY COM 2017-04-10 5 VAN DODGE BUS LANE VIOLATION
4006265037 FZX9232 NY PAS 2016-08-23 5 SUBN FORD BUS LANE VIOLATION
4006535600 N203399C NY OMT 2016-10-19 5 SUBN FORD BUS LANE VIOLATION
4007156700 92163MG NY COM 2017-04-13 5 VAN FRUEH BUS LANE VIOLATION
4006687989 MIQ600 SC PAS 2016-11-21 5 VN HONDA BUS LANE VIOLATION
4006943052 2AE3984 MD PAS 2017-02-01 5 SW LINCO BUS LANE VIOLATION
4007306795 HLG4926 NY PAS 2017-05-30 5 SUBN TOYOT BUS LANE VIOLATION
4007124590 T715907C NY OMT 2017-04-03 5 SUBN TOYOT BUS LANE VIOLATION
5096061966 HRC9475 NY PAS 2017-04-18 7 SUBN CADIL FAILURE TO STOP AT RED LIGHT
5094070400 DYP8042 NY PAS 2016-10-26 7 SUBN CHEVR FAILURE TO STOP AT RED LIGHT
5094906770 G30ESY NJ PAS 2017-01-01 7 WAGO CHRYS FAILURE TO STOP AT RED LIGHT
5093319363 GGT8868 NY PAS 2016-09-06 7 SUBN CHRYS FAILURE TO STOP AT RED LIGHT
5092638

8358531096 FGN4109 NY PAS 2016-11-14 20 SUBN CHRYS 20A-No Parking (Non-COM)
7623299300 530LAW FL PAS 2016-07-23 20 SUBN CITRO 20A-No Parking (Non-COM)
1416638830 GLP367 NY PAS 2017-04-30 20 SUBN DODGE 
7441946736 GMT8049 NY PAS 2017-02-27 20 4DSD DODGE 20A-No Parking (Non-COM)
8522913341 KHB4412 PA PAS 2017-04-17 20 4DSD DODGE 20A-No Parking (Non-COM)
8421057479 64921JT NY COM 2017-01-27 20 VAN FORD 20-No Parking (Com Plate)
8533873610 HBZ9785 NY PAS 2017-05-16 20 2DSD FORD 20A-No Parking (Non-COM)
8388024498 HGF2284 NY PAS 2016-11-14 20 SUBN FORD 20A-No Parking (Non-COM)
8086541423 K78FFL NJ PAS 2016-09-07 20 SUBN FORD 20A-No Parking (Non-COM)
8521664060 XW347T NJ PAS 2017-06-09 20 VAN FORD 20-No Parking (Com Plate)
7541979089 GPW5892 NY PAS 2017-01-16 20 SUBN GMC 20A-No Parking (Non-COM)
7487986524 AVS4503 NY PAS 2016-09-16 20 PICK HONDA 20A-No Parking (Non-COM)
8141015345 FRJ7858 NY PAS 2016-07-22 20 SUBN HONDA 20A-No Parking (Non-COM)
8543009080 GJA1784 NY PAS 2017-06-02 20 SUBN HO

8561603562 GUM5490 NY PAS 2017-06-08 21 SUBN SUBAR 21-No Parking (street clean)
8260034475 HNR3840 NY 999 2017-01-23 21 4DSD SUBAR 21-No Parking (street clean)
8489005175 450SEY CT PAS 2016-12-29 21 SUBN TOYOT 21-No Parking (street clean)
8327056311 809RGE FL PAS 2017-01-11 21 SUBN TOYOT 21-No Parking (street clean)
8035390004 853NG3 MA PAS 2016-09-29 21 4DSD TOYOT 21-No Parking (street clean)
8473630373 923156N NJ PAS 2017-03-21 21 SUBN TOYOT 21-No Parking (street clean)
7431960360 ECL2156 NY PAS 2016-07-19 21 SUBN TOYOT 21-No Parking (street clean)
1411163175 EFY2042 NY PAS 2016-07-14 21 SUBN TOYOT 
8293532713 EGE9369 NY PAS 2016-07-29 21 SUBN TOYOT 21-No Parking (street clean)
8554703479 EPD3783 NY PAS 2017-05-04 21 4DSD TOYOT 21-No Parking (street clean)
8444566147 FJB1476 NY PAS 2017-02-22 21 4DSD TOYOT 21-No Parking (street clean)
1412635536 GLS6347 NY PAS 2016-12-12 21 SUBN TOYOT 
8526761936 GMZ4346 NY PAS 2017-05-02 21 SUBN TOYOT 21-No Parking (street clean)
7061109026 GWE6421 

4632906438 GTB2097 NY PAS 2017-04-24 36 SUBN ME/BE PHTO SCHOOL ZN SPEED VIOLATION
4633747381 HPA3069 NY PAS 2017-05-12 36 2DSD MINI PHTO SCHOOL ZN SPEED VIOLATION
4628411931 GTJ9563 NY PAS 2016-12-16 36 4DSD MITSU PHTO SCHOOL ZN SPEED VIOLATION
4622724601 EJG3090 NY PAS 2016-08-04 36 4DSD NISSA PHTO SCHOOL ZN SPEED VIOLATION
4624182546 FHD2704 NY PAS 2016-09-27 36 4DSD NISSA PHTO SCHOOL ZN SPEED VIOLATION
4630154796 FYB4398 NY PAS 2017-01-31 36 SUBN NISSA PHTO SCHOOL ZN SPEED VIOLATION
4633052056 GAU8752 NY PAS 2017-04-27 36 SUBN NISSA PHTO SCHOOL ZN SPEED VIOLATION
4625956213 GFM3693 NY PAS 2016-10-27 36 4DSD NISSA PHTO SCHOOL ZN SPEED VIOLATION
4634991421 GMS3982 NY PAS 2017-06-13 36 SUBN NISSA PHTO SCHOOL ZN SPEED VIOLATION
4625626006 GTL5801 NY PAS 2016-10-26 36 4DSD NISSA PHTO SCHOOL ZN SPEED VIOLATION
4624010073 GUU9995 NY PAS 2016-09-23 36 SUBN NISSA PHTO SCHOOL ZN SPEED VIOLATION
4630535779 HKM3744 NY PAS 2017-02-03 36 4DSD NISSA PHTO SCHOOL ZN SPEED VIOLATION
4633458899 HMX916

8476754036 5733VQ ME PAS 2017-01-28 40 4DSD CHEVR 40-Fire Hydrant
8524758790 62122MJ NY COM 2017-04-14 40 VAN CHEVR 40-Fire Hydrant
8090532962 EUZ1312 NY PAS 2017-04-11 40 4DSD DODGE 40-Fire Hydrant
7807866536 10992ME NY COM 2016-11-14 40 VAN FORD 40-Fire Hydrant
8521411923 AH20097 AZ PAS 2017-04-22 40 VAN FORD 40-Fire Hydrant
8486983800 C087382 CT PAS 2017-04-10 40 VAN FORD 40-Fire Hydrant
7813974049 GLS7002 NY PAS 2016-09-09 40 SUBN FORD 40-Fire Hydrant
8526650464 GXA7604 NY PAS 2017-02-24 40 2DSD FORD 40-Fire Hydrant
7509978221 12127MJ NY COM 2017-02-27 40 VAN FRUEH 40-Fire Hydrant
8521702024 67667MH NY COM 2017-02-22 40 DELV FRUEH 40-Fire Hydrant
8503070157 85849MA NY COM 2017-04-05 40 DELV FRUEH 40-Fire Hydrant
8100540172 88700JA NY COM 2017-02-23 40 DELV FRUEH 40-Fire Hydrant
8530477870 17623MK NY COM 2017-06-12 40 VAN GMC 40-Fire Hydrant
8553659872 36012JH NY COM 2017-06-05 40 VAN GMC 40-Fire Hydrant
8283027748 FLJ1982 TX PAS 2016-07-29 40 4DSD HONDA 40-Fire Hydrant
8533207992 F

8471891440 42530MG NY COM 2017-02-28 71 DELV CHEVR 71A-Insp Sticker Expired (NYS)
8555150875 ETF9022 NY PAS 2017-05-02 71 SUBN CHEVR 71A-Insp Sticker Expired (NYS)
8188582906 GMT8516 NY PAS 2016-11-01 71 PICK CHEVR 71A-Insp Sticker Expired (NYS)
8206527321 HCD4100 NY PAS 2016-11-01 71 SUBN CHEVR 71A-Insp Sticker Expired (NYS)
8519013259 HNY7165 NY PAS 2017-05-16 71 SUBN CHEVR 71-Insp. Sticker Missing (NYS
8145014311 HJV3061 NY PAS 2016-12-13 71 SUBN CHRYS 71A-Insp Sticker Expired (NYS)
8548503257 GWB7005 NY PAS 2017-05-01 71 4DSD DODGE 71A-Insp Sticker Expired (NYS)
8235044468 FXX6545 NY PAS 2016-10-19 71 SUBN FORD 71A-Insp Sticker Expired (NYS)
7491184224 GAM6902 NY PAS 2017-04-07 71 SUBN FORD 71A-Insp Sticker Expired (NYS)
8559052999 GMR3386 NY PAS 2017-05-13 71 SUBN FORD 71A-Insp Sticker Expired (NYS)
8514728416 GYP8889 NY PAS 2017-05-28 71 SUBN FORD 71A-Insp Sticker Expired (NYS)
7162362357 GCE5560 NY PAS 2016-08-05 71 2DSD HONDA 71A-Insp Sticker Expired (NYS)
8486910511 GXC5916 NY

In [25]:
from collections import defaultdict

In [26]:
counts = defaultdict(int)
for i in parsed_data():
    counts[i.vehicle_make] =+ 1
for make, count in sorted(mapping.items(),key=lambda t: t[1],reverse=True):
    print(make,count)

4006478550 VAD7274 VA PAS 2016-10-05 5 4D BMW BUS LANE VIOLATION
4006462396 22834JK NY COM 2016-09-30 5 VAN CHEVR BUS LANE VIOLATION
4007117810 21791MG NY COM 2017-04-10 5 VAN DODGE BUS LANE VIOLATION
4006265037 FZX9232 NY PAS 2016-08-23 5 SUBN FORD BUS LANE VIOLATION
4006535600 N203399C NY OMT 2016-10-19 5 SUBN FORD BUS LANE VIOLATION
4007156700 92163MG NY COM 2017-04-13 5 VAN FRUEH BUS LANE VIOLATION
4006687989 MIQ600 SC PAS 2016-11-21 5 VN HONDA BUS LANE VIOLATION
4006943052 2AE3984 MD PAS 2017-02-01 5 SW LINCO BUS LANE VIOLATION
4007306795 HLG4926 NY PAS 2017-05-30 5 SUBN TOYOT BUS LANE VIOLATION
4007124590 T715907C NY OMT 2017-04-03 5 SUBN TOYOT BUS LANE VIOLATION
5096061966 HRC9475 NY PAS 2017-04-18 7 SUBN CADIL FAILURE TO STOP AT RED LIGHT
5094070400 DYP8042 NY PAS 2016-10-26 7 SUBN CHEVR FAILURE TO STOP AT RED LIGHT
5094906770 G30ESY NJ PAS 2017-01-01 7 WAGO CHRYS FAILURE TO STOP AT RED LIGHT
5093319363 GGT8868 NY PAS 2016-09-06 7 SUBN CHRYS FAILURE TO STOP AT RED LIGHT
5092638

8330511935 HBY7509 NY PAS 2016-07-30 19 4DSD TOYOT 19-No Stand (bus stop)
1423536058 HCS2336 NY PAS 2017-01-27 19 SUBN TOYOT 
8519016595 QU655 RI PAS 2017-06-12 19 4DSD TOYOT 19-No Stand (bus stop)
8530522310 T640396C NY OMT 2017-05-27 19 4DSD TOYOT 19-No Stand (bus stop)
8269048434 CDPE57 FL PAS 2016-08-21 20 4DSD ACURA 20A-No Parking (Non-COM)
7036998684 VML8110 VA PAS 2017-02-21 20 4DSD BMW 20A-No Parking (Non-COM)
8515215408 HFB1731 NY PAS 2017-04-14 20 SUBN CADIL 20A-No Parking (Non-COM)
8470607042 MYSUMMER NY OMT 2016-07-27 20 SUBN CADIL 20A-No Parking (Non-COM)
1371960501 FEP2455 NY PAS 2016-07-18 20 SUBN CHEVR 
8550963045 GSU8031 NY PAS 2017-06-20 20 4DSD CHEVR 20A-No Parking (Non-COM)
8358531096 FGN4109 NY PAS 2016-11-14 20 SUBN CHRYS 20A-No Parking (Non-COM)
7623299300 530LAW FL PAS 2016-07-23 20 SUBN CITRO 20A-No Parking (Non-COM)
1416638830 GLP367 NY PAS 2017-04-30 20 SUBN DODGE 
7441946736 GMT8049 NY PAS 2017-02-27 20 4DSD DODGE 20A-No Parking (Non-COM)
8522913341 KHB4412 

8523487694 D03SU FL PAS 2017-06-12 21 SUBN HONDA 21-No Parking (street clean)
7803577878 DRU6182 NY PAS 2016-07-19 21 SUBN HONDA 21-No Parking (street clean)
8551867635 EJY9809 NY PAS 2017-06-15 21 SUBN HONDA 21-No Parking (street clean)
8177030073 FPN6798 NY PAS 2016-07-22 21 SUBN HONDA 21-No Parking (street clean)
7522710777 FTJ4934 NY PAS 2016-08-02 21 4DSD HONDA 21-No Parking (street clean)
1415914771 FWY7687 NY PAS 2016-11-05 21 SUBN HONDA 
8511777581 GBP8071 NY PAS 2017-06-23 21 4DSD HONDA 21-No Parking (street clean)
8507224441 GHR1050 NY PAS 2017-06-12 21 SUBN HONDA 21-No Parking (street clean)
1418138575 GML6346 NY PAS 2016-12-19 21 SDN HONDA 
8494361650 GPR1060 NY PAS 2017-01-19 21 4DSD HONDA 21-No Parking (street clean)
8532805115 GRH7130 NY PAS 2017-03-30 21 SUBN HONDA 21-No Parking (street clean)
1420074817 GYX5626 NY PAS 2017-03-23 21 SUBN HONDA 
8081038309 HGU5174 NY PAS 2016-07-13 21 4DSD HONDA 21-No Parking (street clean)
8539658732 HLY9761 NY PAS 2017-04-20 21 SUBN HO

1410027223 ENS9253 NY PAS 2016-10-31 24 SDN VOLKS 
8479824712 58910MG NY COM 2017-04-21 26 DELV ISUZU 26-No Stnd (for-hire veh only)
1416527722 AHG9422 NY PAS 2017-01-05 27 SUBN TOYOT 
8512270482 Y26CRJ NJ PAS 2017-06-03 31 SUBN BSA 31-No Stand (Com. Mtr. Zone)
8498706890 THANEDAR NY OMT 2017-03-22 31 SUBN CADIL 31-No Stand (Com. Mtr. Zone)
8328546632 HHP7446 NY PAS 2016-12-02 31 2DSD CHEVR 31-No Stand (Com. Mtr. Zone)
8496254203 GMW3411 NY PAS 2017-05-26 31 PICK DODGE 31-No Stand (Com. Mtr. Zone)
8552002887 GTP5204 NY PAS 2017-04-24 31 PICK DODGE 31-No Stand (Com. Mtr. Zone)
8480300255 GHG9797 NY PAS 2016-11-18 31 4DSD HYUND 31-No Stand (Com. Mtr. Zone)
8442517080 KCD4692 PA PAS 2016-10-06 31 4DSD INFIN 31-No Stand (Com. Mtr. Zone)
8482450256 GGN8817 NY PAS 2016-11-18 31 4DSD LEXUS 31-No Stand (Com. Mtr. Zone)
8480624486 987536N NJ PAS 2017-06-19 31 SUBN ME/BE 31-No Stand (Com. Mtr. Zone)
7059132030 HLC5085 NY PAS 2017-02-06 31 4DSD NISSA 31-No Stand (Com. Mtr. Zone)
8482371824 Z31782

4633257055 HJN6853 NY PAS 2017-05-02 36 SUBN LEXUS PHTO SCHOOL ZN SPEED VIOLATION
4629197515 N78FLU NJ PAS 2017-01-09 36 WAGO LEXUS PHTO SCHOOL ZN SPEED VIOLATION
4633226356 PTJ969 NY PAS 2017-05-01 36 SUBN LEXUS PHTO SCHOOL ZN SPEED VIOLATION
4626735174 HGR5863 NY PAS 2016-11-14 36 4DSD LINCO PHTO SCHOOL ZN SPEED VIOLATION
4622247501 T635722C NY OMT 2016-07-13 36 4DSD LINCO PHTO SCHOOL ZN SPEED VIOLATION
4626383063 T701050C NY OMT 2016-11-04 36 4DSD LINCO PHTO SCHOOL ZN SPEED VIOLATION
4631263763 92814 NY MED 2017-03-03 36 4DSD ME/BE PHTO SCHOOL ZN SPEED VIOLATION
4626546109 39999MB NY COM 2016-11-07 36 VAN ME/BE PHTO SCHOOL ZN SPEED VIOLATION
4626838832 EVV1706 NY PAS 2016-11-16 36 2DSD ME/BE PHTO SCHOOL ZN SPEED VIOLATION
4623995203 FHV5859 NY PAS 2016-09-23 36 4DSD ME/BE PHTO SCHOOL ZN SPEED VIOLATION
4632906438 GTB2097 NY PAS 2017-04-24 36 SUBN ME/BE PHTO SCHOOL ZN SPEED VIOLATION
4633747381 HPA3069 NY PAS 2017-05-12 36 2DSD MINI PHTO SCHOOL ZN SPEED VIOLATION
4628411931 GTJ9563 N

8355032767 96143MB NY COM 2016-12-08 38 VAN CHEVR 38-Failure to Display Muni Rec
8549906335 FCM7111 NY PAS 2017-05-24 38 SUBN CHEVR 38-Failure to Display Muni Rec
7034978219 GHK9710 NY PAS 2016-09-21 38 SUBN CHEVR 38-Failure to Display Muni Rec
8543015133 HFV7703 NY PAS 2017-06-20 38 SUBN CHEVR 38-Failure to Display Muni Rec
8490761784 HHS5227 NY PAS 2017-01-20 38 SUBN CHEVR 38-Failure to Display Muni Rec
8206046343 HMU5912 NY PAS 2017-03-04 38 SUBN CHEVR 38-Failure to Display Muni Rec
8491610029 XDDX12 NJ PAS 2017-01-21 38 VAN CHEVR 38-Failure to Display Muni Rec
8548102378 HEJ9518 NY PAS 2017-04-26 38 SUBN CHRYS 38-Failure to Display Muni Rec
8550958402 HHU8104 NY PAS 2017-05-24 38 SUBN DODGE 38-Failure to Display Muni Rec
8523051284 SH3ZBAD AL PAS 2017-03-11 38 4DSD DODGE 38-Failure to Display Muni Rec
8470577803 W468KH IA PAS 2017-02-04 38 PICK DODGE 38-Failure to Display Muni Rec
8063538547 26013KA NY COM 2016-08-04 38 VAN FORD 38-Failure to Display Muni Rec
7038676805 29518MD NY 

8090532962 EUZ1312 NY PAS 2017-04-11 40 4DSD DODGE 40-Fire Hydrant
7807866536 10992ME NY COM 2016-11-14 40 VAN FORD 40-Fire Hydrant
8521411923 AH20097 AZ PAS 2017-04-22 40 VAN FORD 40-Fire Hydrant
8486983800 C087382 CT PAS 2017-04-10 40 VAN FORD 40-Fire Hydrant
7813974049 GLS7002 NY PAS 2016-09-09 40 SUBN FORD 40-Fire Hydrant
8526650464 GXA7604 NY PAS 2017-02-24 40 2DSD FORD 40-Fire Hydrant
7509978221 12127MJ NY COM 2017-02-27 40 VAN FRUEH 40-Fire Hydrant
8521702024 67667MH NY COM 2017-02-22 40 DELV FRUEH 40-Fire Hydrant
8503070157 85849MA NY COM 2017-04-05 40 DELV FRUEH 40-Fire Hydrant
8100540172 88700JA NY COM 2017-02-23 40 DELV FRUEH 40-Fire Hydrant
8530477870 17623MK NY COM 2017-06-12 40 VAN GMC 40-Fire Hydrant
8553659872 36012JH NY COM 2017-06-05 40 VAN GMC 40-Fire Hydrant
8283027748 FLJ1982 TX PAS 2016-07-29 40 4DSD HONDA 40-Fire Hydrant
8533207992 FMF1924 NY PAS 2017-03-28 40 SUBN HONDA 40-Fire Hydrant
8525513556 GBE4428 NY PAS 2017-04-17 40 4DSD HONDA 40-Fire Hydrant
8490855328

8528912840 HNY5206 NY PAS 2017-04-21 74 SUBN GMC 74A-Improperly Displayed Plate
8511501113 FBP6836 NY PAS 2017-02-16 74 SUBN HONDA 74-Missing Display Plate
8276551890 HJR4081 NY PAS 2017-01-07 74 SUBN HONDA 74A-Improperly Displayed Plate
8214550531 HJY2207 NY PAS 2017-03-09 74 4DSD HYUND 74-Missing Display Plate
8487469012 27527ME NY COM 2017-01-27 74 DELV INTER 74-Missing Display Plate
8034780236 HCD3158 NY PAS 2016-09-02 74 SUBN KIA 74A-Improperly Displayed Plate
8545052455 HMK1149 NY PAS 2017-06-07 74 4DSD ME/BE 74-Missing Display Plate
1411263467 BLANKPLATE 99 999 2017-02-13 74 SDN NISSA 
8540001550 HLA4803 NY PAS 2017-03-23 74 4DSD SAAB 74-Missing Display Plate
8556155431 HFB9919 NY PAS 2017-05-26 75 4DSD DODGE 75-No Match-Plate/Reg. Sticker
8394016790 31690BB NY OMR 2016-09-06 77 BUS AM/T 77-Parked Bus (exc desig area)
1419472306 56090BA NY OMR 2017-02-20 77 BUS INTER 
8363508688 97256 MA APP 2016-08-13 77 BUS NS/OT 77-Parked Bus (exc desig area)
1407740258 2513JMG NY COM 2017-01

In [29]:
def violation_by_make():
    counts = defaultdict(int)
    for i in parsed_data():
        counts[i.vehicle_make] += 1
    return {make:cnt for make, cnt in sorted(counts.items(),
                                            key = lambda t: t[1],
                                            reverse = True)}

In [30]:
violation_by_make()

4006478550 VAD7274 VA PAS 2016-10-05 5 4D BMW BUS LANE VIOLATION
4006462396 22834JK NY COM 2016-09-30 5 VAN CHEVR BUS LANE VIOLATION
4007117810 21791MG NY COM 2017-04-10 5 VAN DODGE BUS LANE VIOLATION
4006265037 FZX9232 NY PAS 2016-08-23 5 SUBN FORD BUS LANE VIOLATION
4006535600 N203399C NY OMT 2016-10-19 5 SUBN FORD BUS LANE VIOLATION
4007156700 92163MG NY COM 2017-04-13 5 VAN FRUEH BUS LANE VIOLATION
4006687989 MIQ600 SC PAS 2016-11-21 5 VN HONDA BUS LANE VIOLATION
4006943052 2AE3984 MD PAS 2017-02-01 5 SW LINCO BUS LANE VIOLATION
4007306795 HLG4926 NY PAS 2017-05-30 5 SUBN TOYOT BUS LANE VIOLATION
4007124590 T715907C NY OMT 2017-04-03 5 SUBN TOYOT BUS LANE VIOLATION
5096061966 HRC9475 NY PAS 2017-04-18 7 SUBN CADIL FAILURE TO STOP AT RED LIGHT
5094070400 DYP8042 NY PAS 2016-10-26 7 SUBN CHEVR FAILURE TO STOP AT RED LIGHT
5094906770 G30ESY NJ PAS 2017-01-01 7 WAGO CHRYS FAILURE TO STOP AT RED LIGHT
5093319363 GGT8868 NY PAS 2016-09-06 7 SUBN CHRYS FAILURE TO STOP AT RED LIGHT
5092638

7606849481 T638216C NY OMT 2017-05-08 20 4DSD TOYOT 20A-No Parking (Non-COM)
8293037179 GJU4869 NY PAS 2016-08-26 21 4DSD ACURA 21-No Parking (street clean)
8009559234 HBU6907 NY PAS 2016-08-10 21 4DSD ACURA 21-No Parking (street clean)
8545156546 EVM6598 NY PAS 2017-05-03 21 4DSD AUDI 21-No Parking (street clean)
8516050920 GXY1094 NY PAS 2017-03-03 21 SUBN AUDI 21-No Parking (street clean)
8544889610 VVP9002 VA PAS 2017-06-14 21 4DSD AUDI 21-No Parking (street clean)
8318510150 99H1010 TN PAS 2016-08-02 21 2DSD BMW 21-No Parking (street clean)
8491819083 FHY6755 NY PAS 2017-03-03 21 4DSD BMW 21-No Parking (street clean)
7725863710 GNF2877 NY PAS 2016-09-26 21 4DSD BMW 21-No Parking (street clean)
1412956201 GNR5598 NY PAS 2016-07-20 21 SDN BMW 
7919376452 HAS5112 NY PAS 2017-01-27 21 SUBN BMW 21-No Parking (street clean)
7970393330 HAT6686 NY PAS 2016-08-17 21 4DSD BMW 21-No Parking (street clean)
8537965042 HFP5029 NY PAS 2017-05-16 21 4DSD BMW 21-No Parking (street clean)
858115393

8566704320 L62FLC NJ PAS 2017-05-17 24 SUBN CHEVR 24-No Parking (exc auth veh)
8536961156 X92FSM NJ PAS 2017-03-28 24 SUBN HYUND 24-No Parking (exc auth veh)
1408122029 DSN6323 NY PAS 2016-07-04 24 SUBN JEEP 
8266568753 F12GRE NJ PAS 2016-11-11 24 SUBN JEEP 24-No Parking (exc auth veh)
8446524806 GYG8911 NY PAS 2016-12-18 24 SUBN JEEP 24-No Parking (exc auth veh)
8316519355 HDJ7785 PA PAS 2016-07-14 24 2DSD NISSA 24-No Parking (exc auth veh)
8533035640 T687600C NY OMT 2017-05-31 24 4DSD NISSA 24-No Parking (exc auth veh)
8526732663 ASV2478 NY PAS 2017-06-22 24 4DSD TOYOT 24-No Parking (exc auth veh)
1410027223 ENS9253 NY PAS 2016-10-31 24 SDN VOLKS 
8479824712 58910MG NY COM 2017-04-21 26 DELV ISUZU 26-No Stnd (for-hire veh only)
1416527722 AHG9422 NY PAS 2017-01-05 27 SUBN TOYOT 
8512270482 Y26CRJ NJ PAS 2017-06-03 31 SUBN BSA 31-No Stand (Com. Mtr. Zone)
8498706890 THANEDAR NY OMT 2017-03-22 31 SUBN CADIL 31-No Stand (Com. Mtr. Zone)
8328546632 HHP7446 NY PAS 2016-12-02 31 2DSD CHEVR

8424504914 53185JX NY COM 2016-08-15 37 VAN FORD 37-Expired Muni Meter
8512119433 76877MJ NY COM 2017-05-24 37 VAN FORD 37-Expired Muni Meter
8513111030 GMZ9316 NY PAS 2017-04-26 37 SUBN FORD 37-Expired Muni Meter
8095027893 GVU9201 NY PAS 2016-08-09 37 4DSD FORD 37-Expired Muni Meter
7018187588 J96GMD NJ PAS 2016-11-12 37 PICK FORD 37-Expired Muni Meter
8519703471 Y81GLL NJ PAS 2017-02-24 37 VAN FORD 37-Expired Muni Meter
8519703094 26684MA NY COM 2017-02-23 37 VAN FRUEH 37-Expired Muni Meter
8081576769 19662JA NY COM 2016-12-19 37 VAN GMC 37-Expired Muni Meter
8323007330 MSINGH NY SRF 2016-08-22 37 SUBN GMC 37-Expired Muni Meter
8522218316 EFJ1721 NY PAS 2017-05-17 37 SUBN HONDA 37-Expired Muni Meter
8362534679 FLW5550 NY PAS 2017-04-06 37 SUBN HONDA 37-Expired Muni Meter
7816795701 GND8994 NY PAS 2016-11-05 37 4DSD HONDA 37-Expired Muni Meter
8188109824 GVK6209 NY PAS 2016-12-10 37 SUBN HONDA 37-Expired Muni Meter
8553153472 HDN9923 NY PAS 2017-05-11 37 SUBN HONDA 37-Expired Muni Me

8558750191 GSH2040 NY PAS 2017-04-20 46 4DSD ACURA 46A-Double Parking (Non-COM)
8502856637 GGL9484 NY PAS 2017-02-10 46 4DSD AUDI 46A-Double Parking (Non-COM)
1420319401 KKN4822 PA PAS 2017-05-15 46 SUBN BUICK 
7067929763 17626MK NY COM 2017-01-31 46 VAN CHEVR 46B-Double Parking (Com-100Ft)
1387522700 71017JM NY COM 2016-08-25 46 VAN CHEVR 
8008621527 HFP7192 NY PAS 2016-10-25 46 VAN CHEVR 46A-Double Parking (Non-COM)
1413687738 25570MC NY COM 2016-09-06 46 DELV DODGE 
8513917291 33147MK NY COM 2017-06-27 46 VAN DODGE 46A-Double Parking (Non-COM)
8505564479 99828MC NY COM 2017-03-07 46 VAN DODGE 46B-Double Parking (Com-100Ft)
8513554716 XWD5801 VA PAS 2017-03-03 46 SUBN DODGE 46A-Double Parking (Non-COM)
1420122800 AB80443 CT PAS 2017-03-13 46 SDN FIAT 
8472012591 48768JZ NY COM 2016-11-09 46 VAN FORD 46B-Double Parking (Com-100Ft)
1401381406 EUM7025 NY PAS 2016-11-14 46 SUBN FORD 
1417599716 12203MG NY COM 2016-12-01 46 VAN FRUEH 
8019072664 12211MG NY COM 2016-11-11 46 VAN FRUEH 46B-

{'TOYOT': 112,
 'HONDA': 106,
 'FORD': 104,
 'CHEVR': 76,
 'NISSA': 70,
 'DODGE': 45,
 'FRUEH': 44,
 'ME/BE': 38,
 'GMC': 35,
 'HYUND': 35,
 'BMW': 34,
 'LEXUS': 26,
 'INTER': 25,
 'JEEP': 22,
 'NS/OT': 18,
 'SUBAR': 18,
 'INFIN': 13,
 'LINCO': 12,
 'CHRYS': 12,
 'ACURA': 12,
 'AUDI': 12,
 'VOLVO': 12,
 'MITSU': 11,
 'ISUZU': 10,
 'CADIL': 9,
 'KIA': 8,
 'VOLKS': 8,
 'HIN': 6,
 'KENWO': 5,
 'ROVER': 5,
 'BUICK': 5,
 'MAZDA': 5,
 'MERCU': 4,
 'JAGUA': 3,
 'SMART': 3,
 'PORSC': 3,
 'WORKH': 2,
 'SATUR': 2,
 'SCION': 2,
 'SAAB': 2,
 'HINO': 2,
 'FIR': 1,
 'OLDSM': 1,
 'PETER': 1,
 'CITRO': 1,
 'GEO': 1,
 'YAMAH': 1,
 'BSA': 1,
 'MINI': 1,
 'PONTI': 1,
 'SPRI': 1,
 'PLYMO': 1,
 'UPS': 1,
 'FIAT': 1,
 'UD': 1,
 'UTILI': 1,
 'GMCQ': 1,
 'STAR': 1,
 'AM/T': 1,
 'MI/F': 1}